# Heat transfer simulation using Physics-Informed Neural Network

In [ ]:
import sys
sys.path.append('./src')

In [ ]:
from heat_solver import HeatForwardSolver
import plotter
from params import u0, options, boundaries
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm
from pinn import PINN
import data_generation as dg
from predict import predict
from pprint import pprint

## Numerical simulation using FDM

In [ ]:
solver = HeatForwardSolver(**options)
solver.set_initial(u0)
solver.set_boundaries(boundaries)

solver.solve()

plotter.animate_plot(solver.u, solver.delta_t, filename='plots/numerical_prediction.gif')
plt = plotter.plot_frame(solver.u, solver.u.shape[2]-1, solver.delta_t)
plt.show()

pprint(options)

## PINN traning and prediciton

In [ ]:
device = device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

scaled_alpha = dg.scale_alpha(options['alpha'])

### Traning with COLLOCATION points ONLY

In [ ]:
pinn = PINN([3, 40, 40, 40, 40, 40, 1], device, scaled_alpha)

traning_data = dg.generate_traning_data(device, n_colloc=10_000)

pinn.start_train(traning_data, 20_000)
plotter.plot_loss(pinn.loss_history, pinn.epoch_history).show()

u = predict(pinn)

plotter.animate_plot(u, options['delta_t'], filename="plots/pinn_prediction_colloc.gif")
plt = plotter.plot_frame(u, u.shape[2]-1, options['delta_t'])
plt.show()

### Traning with collocation points, BC and IC

In [ ]:
pinn = PINN([3, 40, 40, 40, 40, 40, 1], device, scaled_alpha)

traning_data = dg.generate_traning_data(device, n_colloc=10_000, n_bc=2000, n_ic=800)

pinn.start_train(traning_data, 20_000)
plotter.plot_loss(pinn.loss_history, pinn.epoch_history).show()

u = predict(pinn)

plotter.animate_plot(u, options['delta_t'], filename="plots/pinn_prediction_pde.gif")
plt = plotter.plot_frame(u, u.shape[2]-1, options['delta_t'])
plt.show()

### Training with DATA points ONLY

In [ ]:
pinn = PINN([3, 40, 40, 40, 40, 40, 1], device, scaled_alpha)

traning_data = dg.generate_traning_data(device, u=solver.u, n_data=20_000)

pinn.start_train(traning_data, 20_000)
plotter.plot_loss(pinn.loss_history, pinn.epoch_history).show()

u = predict(pinn)

plotter.animate_plot(u, options['delta_t'], filename="plots/pinn_prediction_data.gif")
plt = plotter.plot_frame(u, u.shape[2]-1, options['delta_t'])
plt.show()

### Training with DATA and PDE (collocation points, BC and IC)

In [ ]:
pinn = PINN([3, 40, 40, 40, 40, 40, 1], device, scaled_alpha)

traning_data = dg.generate_traning_data(device, u=solver.u, n_data=20_000, n_colloc=10_000, n_bc=2000, n_ic=800)

pinn.start_train(traning_data, 20_000)
plotter.plot_loss(pinn.loss_history, pinn.epoch_history).show()

u = predict(pinn)

plotter.animate_plot(u, options['delta_t'], filename="plots/pinn_prediction_all.gif")
plt = plotter.plot_frame(u, u.shape[2]-1, options['delta_t'])
plt.show()